# Assignment 1 - INSYS661
 



#### Name : Arham Anwar 

#### McGill ID: 

#### Class: 76

In [115]:
# import libraries

import pandas as pd
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

#### Data Preprocessing Functions

In [116]:
# Functions created for preprocessing [ For the ease of grader, this section can be skipped ]



import pandas as pd
from IPython.display import display

import pandas as pd
from IPython.display import display

def col_stats(df):
    """
    Display statistics for each column in a DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame.

    Returns:
    None
    """
    # Initialize an empty list to store statistics
    statistics = []

    # Iterate through each column in the DataFrame
    for column in df.columns:
        unique_count = df[column].nunique()
        data_type = df[column].dtype

        if pd.api.types.is_numeric_dtype(df[column]):
            mean_value = df[column].mean()
            median_value = df[column].median()
            mode_value = df[column].mode().values[0] if not df[column].empty else None
            min_value = df[column].min()
            max_value = df[column].max()
            missing_count = df[column].isnull().sum()
            total_rows = len(df)
            sum_value = df[column].sum()
        else:
            mean_value = 'N/A'
            median_value = 'N/A'
            mode_value = 'N/A'
            min_value = 'N/A'
            max_value = 'N/A'
            missing_count = 'N/A'
            total_rows = len(df)
            sum_value = 'N/A'

        # Append statistics for the column to the list
        statistics.append([column, unique_count, data_type, mean_value, median_value,
                           mode_value, min_value, max_value, missing_count, total_rows, sum_value])

    # Create a DataFrame for the statistics
    statistics_df = pd.DataFrame(statistics, columns=['Column', 'Unique Count', 'Data Type', 'Mean', 'Median',
                                                      'Mode', 'Min', 'Max', 'Missing Count', 'Total Rows', 'Sum'])

    # Display the DataFrame
    display(statistics_df)





# missing values

def remove_missing_values(df):
    # Count rows before removing missing values
    count_before = len(df)

    # Remove rows with missing values
    df.dropna(inplace=True)

    # Count rows after removing missing values
    count_after = len(df)

    print("Count of rows before removing missing values:", count_before)
    print("Count of rows after removing missing values:", count_after)

    return df


# add index

def add_index_to_dataframe(df, column_name=None):
    """
    Adds an index to the DataFrame either using an existing column or by creating a new numerical index.

    Parameters:
    df (pd.DataFrame): Input DataFrame.
    column_name (str): Name of the column to use as the index. Default is None (uses default numerical index).

    Returns:
    pd.DataFrame: DataFrame with the specified column as the index or a new numerical index.
    """
    if column_name is None:
        # Use default numerical index
        df_with_index = df.reset_index(drop=True)
    else:
        # Set the specified column as the index
        df_with_index = df.set_index(column_name)

    return df_with_index


# remove duplicates

def remove_dup(df, columns=None):
    """
    Remove duplicates from a DataFrame based on specified columns or the entire row.

    Parameters:
    df (pd.DataFrame): Input DataFrame.
    columns (list): List of column names to consider for identifying duplicates. Default is None (consider entire rows).

    Returns:
    pd.DataFrame: DataFrame with duplicates removed.
    """
    # Display the number of duplicate rows
    duplicate_count = df.duplicated(subset=columns).sum()
    print(f"Number of duplicate rows removed: {duplicate_count}")

    if columns:
        # Display duplicates based on specified columns
        duplicates_df = df[df.duplicated(subset=columns, keep=False)]
        print("Duplicates only (based on specified columns):")
        print(duplicates_df)
        
        # Remove duplicates based on specified columns
        df_cleaned = df.drop_duplicates(subset=columns)
    else:
        # Display duplicates based on entire rows
        duplicates_df = df[df.duplicated(keep=False)]
        print("Duplicates only (based on entire rows):")
        print(duplicates_df)
        
        # Remove duplicates based on entire rows
        df_cleaned = df.drop_duplicates()

    return df_cleaned

# cat to num

def cat_to_num(df, column_name):
    """
    Convert a categorical column to numerical representation using LabelEncoder.

    Parameters:
    df (pd.DataFrame): Input DataFrame.
    column_name (str): Name of the categorical column to convert.

    Returns:
    pd.DataFrame: DataFrame with the categorical column converted to numerical representation.
    """
    # Create a LabelEncoder instance
    label_encoder = LabelEncoder()

    # Fit and transform the specified column
    df[column_name + '_numerical'] = label_encoder.fit_transform(df[column_name])

    return df

# scaling

def scale_dataframe_column(df, column_name, method='min_max'):
    """
    Scale a DataFrame column using specified scaling method (Z-score, Min-Max, or Log).

    Parameters:
    df (pd.DataFrame): Input DataFrame.
    column_name (str): Name of the column to scale.
    method (str): Scaling method to use - 'z_score', 'min_max', or 'log'. Default is 'min_max'.

    Returns:
    pd.DataFrame: DataFrame with the specified column scaled.
    """
    df_copy = df.copy()

    if method == 'z_score':
        # Z-score scaling
        scaler = StandardScaler()
        df_copy[column_name + '_scaled'] = scaler.fit_transform(df_copy[[column_name]])

    elif method == 'min_max':
        # Min-Max scaling
        scaler = MinMaxScaler()
        df_copy[column_name + '_scaled'] = scaler.fit_transform(df_copy[[column_name]])

    elif method == 'log':
        # Log scaling (log base 10)
        df_copy[column_name + '_scaled'] = np.log10(df_copy[column_name] + 1)  # Adding 1 to avoid log(0)

    else:
        raise ValueError("Invalid scaling method. Supported methods: 'z_score', 'min_max', 'log'.")

    return df_copy

#### Loading Dataset

In [117]:
Raw = pd.read_csv("/Users/arham/Python Projects/Datasets/ToyotaCorolla.csv")

Toyota = Raw

#### Understanding Data / Raw Data Insights

In [118]:
col_stats(Toyota)

,Column,Unique Count,Data Type,Mean,Median,Mode,Min,Max,Missing Count,Total Rows,Sum
0,Id,1436,int64,721.555014,721.5,1,1,1442,0,1436,1036153
1,Model,319,object,N/A,N/A,N/A,N/A,N/A,N/A,1436,N/A
2,Price,236,int64,10730.824513,9900.0,8950,4350,32500,0,1436,15409464
3,Age_08_04,77,int64,55.947075,61.0,68,1,80,0,1436,80340
4,KM,1263,int64,68533.259749,63389.5,36000,1,243000,0,1436,98413761
5,HP,12,int64,101.502089,110.0,110,69,192,0,1436,145757
6,Automatic,2,int64,0.05571,0.0,0,0,1,0,1436,80
7,CC,13,int64,1576.85585,1600.0,1600,1300,16000,0,1436,2264365
8,Doors,4,int64,4.033426,4.0,5,2,5,0,1436,5792
9,Cylinders,1,int64,4.0,4.0,4,4,4,0,1436,5744


#### Initial insights from raw data

- There are 319 unique models in this dataset of 1436 rows
- The price of these models ranges from $4350 to $32500, with median price as $9900
- Age, KM, HP, Automatic, CC, Doors, Gears and weight appear to be in the right ballpark range with no extreme outliers
- Cylinders column has only one value and thus can be removed from scope for the regression

In [119]:
# Excluding Cylinders
Toyota = Toyota[['Id', 'Model', 'Price', 'Age_08_04', 'KM', 'HP', 'Automatic', 'CC',
       'Doors', 'Gears', 'Weight']]

# Missing Values
Toyota_NA_removed = remove_missing_values(Toyota)

Count of rows before removing missing values: 1436
Count of rows after removing missing values: 1436


/var/folders/2t/c7s0z0zs4698zw0k9pj4f2r80000gn/T/ipykernel_44482/3945833306.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


There are no missing value rows as we can see from the missing rows analysis above

#### Duplicates

In [120]:
# remove duplicates
Toyota_dup_removed = remove_dup(Toyota, columns=['Model', 'Price', 'Age_08_04', 'KM', 'HP', 'Automatic', 'CC',
       'Doors', 'Gears', 'Weight'])

Number of duplicate rows removed: 1
Duplicates only (based on specified columns):
      Id                                       Model  Price  Age_08_04     KM  \
112  114  TOYOTA Corolla VERSO 2.0 D4D LINEA SOL MPV  24950          8  13253   
113  115  TOYOTA Corolla VERSO 2.0 D4D LINEA SOL MPV  24950          8  13253   

      HP  Automatic    CC  Doors  Gears  Weight  
112  116          0  2000      5      5    1320  
113  116          0  2000      5      5    1320  


There was one duplicate row, which has been removed, the duplicate instances can be seen above

In [121]:
Toyota = Toyota_dup_removed
Toyota

,Id,Model,Price,Age_08_04,KM,HP,Automatic,CC,Doors,Gears,Weight
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,46986,90,0,2000,3,5,1165
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,72937,90,0,2000,3,5,1165
2,3,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,41711,90,0,2000,3,5,1165
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,48000,90,0,2000,3,5,1165
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,38500,90,0,2000,3,5,1170
...,...,...,...,...,...,...,...,...,...,...,...
1431,1438,TOYOTA Corolla 1.3 16V HATCHB G6 2/3-Doors,7500,69,20544,86,0,1300,3,5,1025
1432,1439,TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...,10845,72,19000,86,0,1300,3,5,1015
1433,1440,TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...,8500,71,17016,86,0,1300,3,5,1015
1434,1441,TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...,7250,70,16916,86,0,1300,3,5,1015


#### Construct Variables

In [122]:
X = Toyota.iloc[:,3:]
y = Toyota['Price']

#### Detect Multicollinearity

In [123]:
## Detecting multicollinearity
# Create VIF dataframe
from statsmodels.tools.tools import add_constant
X1 = add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X1.columns
  
# Calculating VIF for each feature
from statsmodels.stats.outliers_influence import variance_inflation_factor
for i in range(len(X1.columns)):
    vif_data.loc[vif_data.index[i],"VIF"] = variance_inflation_factor(X1.values, i)

print(vif_data)


     feature          VIF
0      const  1383.423747
1  Age_08_04     1.901144
2         KM     1.678436
3         HP     1.217309
4  Automatic     1.048017
5         CC     1.150257
6      Doors     1.154944
7      Gears     1.109310
8     Weight     1.659263


#### The highest VIF value is for Age

We shouldn't remove any variable as all variables have vif < 5

#### Standardize predictors

In [124]:
## Standardize the predictors
X.describe()

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_X = scaler.fit_transform(X)
scaled_X = pd.DataFrame(scaled_X, columns=X.columns) # transform into a dataframe and add column names

scaled_X


,Age_08_04,KM,HP,Automatic,CC,Doors,Gears,Weight
0,-1.777268,-0.575958,-0.767351,-0.242983,0.998113,-1.084443,-0.140475,1.774964
1,-1.777268,0.116474,-0.767351,-0.242983,0.998113,-1.084443,-0.140475,1.774964
2,-1.723380,-0.716707,-0.767351,-0.242983,0.998113,-1.084443,-0.140475,1.774964
3,-1.615603,-0.548902,-0.767351,-0.242983,0.998113,-1.084443,-0.140475,1.774964
4,-1.400049,-0.802384,-0.767351,-0.242983,0.998113,-1.084443,-0.140475,1.870688
...,...,...,...,...,...,...,...,...
1430,0.701602,-1.281492,-1.034441,-0.242983,-0.651898,-1.084443,-0.140475,-0.905299
1431,0.863267,-1.322689,-1.034441,-0.242983,-0.651898,-1.084443,-0.140475,-1.096747
1432,0.809379,-1.375627,-1.034441,-0.242983,-0.651898,-1.084443,-0.140475,-1.096747
1433,0.755490,-1.378295,-1.034441,-0.242983,-0.651898,-1.084443,-0.140475,-1.096747


#### Train test split

In [148]:
## Cross-Validation  (Validation set approach)
# Separate the data
# Already loaded - > from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size = 0.35, random_state = 662)

#### Linear Regression

In [159]:
# Run linear regression based on the training data
lm2 = LinearRegression()
model2 = lm2.fit(X_train,y_train)

# Generate the prediction value from the test data
y_test_pred = model2.predict(X_test)
# y_test_preddf=pd.DataFrame(y_test_pred, columns=['Predicted rating']) if need to convert to dataframe

# Calculate the MSE for the test set
from sklearn.metrics import mean_squared_error
lm2_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE with simple linear regression = "+str(int(lm2_mse)))


# View results
intercept = model1.intercept_
coefficients = model1.coef_
r2 = model1.score(scaled_X, y)  # Calculate R-squared (R2) score


print("Intercept:", intercept)
print("Coefficients:", coefficients)
print("R-squared (R2) score:", r2)


Test MSE with simple linear regression = 1626924
Intercept: 10720.915679442509
Coefficients: [-2285.15190311  -726.50770423   426.42602248    87.54290447
   -42.05928958    -2.90616303   127.13668496   964.02213644]
R-squared (R2) score: 0.862246229165933


#### Ridge Regression

In [163]:
## 5. Ridge regression
from sklearn.linear_model import Ridge

# Run ridge regression with penalty equals to 1
ridge = Ridge(alpha=1)
ridge_model = ridge.fit(X_train,y_train)

# Print the coefficients
ridge_model.coef_

# Generate the prediction value from the test data
y_test_pred = ridge_model.predict(X_test)

# Calculate the MSE
ridge_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE using ridge with penalty of 1 = "+str(ridge_mse))

print("\n \nLet's test some other values for penalty \n\n")

# Finding optimal penalty
for i in [1,10, 100, 1000, 1000]:
    ridge3 = Ridge(alpha=i)
    ridge_model3 = ridge3.fit(X_train,y_train)
    y_test_pred3 = ridge_model3.predict(X_test)
    print('Alpha = ',i,' / MSE =',int(mean_squared_error(y_test, y_test_pred3)))


Test MSE using ridge with penalty of 1 = 1626677.6254439154

 
Let's test some other values for penalty 


Alpha =  1  / MSE = 1626677
Alpha =  10  / MSE = 1625151
Alpha =  100  / MSE = 1661808
Alpha =  1000  / MSE = 3219985
Alpha =  1000  / MSE = 3219985


#### LASSO

In [171]:
## 6. LASSO
from sklearn.linear_model import Lasso

# Run LASSO with penalty = 1
lasso = Lasso(alpha=1)
lasso_model = lasso.fit(X_train,y_train)

# Generate the prediction value from the test data
y_test_pred4 = lasso_model.predict(X_test)

# Calculate the MSE
lasso_mse = mean_squared_error(y_test, y_test_pred4)
print("Test MSE using lasso with penalty of 1 = "+str(lasso_mse))

# Print the coefficients
print(lasso_model.coef_)


Test MSE using lasso with penalty of 1 = 1627430.565671692
[-2323.58993     -721.55495729   434.74989115    85.36114513
    -3.00538181     0.            80.09478236   910.54914831]


In [166]:

# Finding optimal alpha
for i in [1, 10, 100, 1000, 1000]:
    lasso = Lasso(alpha=i)
    lasso_model = lasso.fit(X_train,y_train)
    y_test_pred5 = lasso_model.predict(X_test)
    print('Alpha = ',i,' / MSE =',int(mean_squared_error(y_test, y_test_pred5)))

Alpha =  1  / MSE = 1627430
Alpha =  10  / MSE = 1631231
Alpha =  100  / MSE = 1685659
Alpha =  1000  / MSE = 3351632
Alpha =  1000  / MSE = 3351632


Test MSE using ridge with penalty of 1 = 1626677.6254439154
Alpha =  10  / MSE = 1625151.5855265579
Alpha =  100  / MSE = 1661808.9820451266
Alpha =  1000  / MSE = 3219985.006691864
Alpha =  1000  / MSE = 3219985.006691864


In [134]:
Toyota.iloc[:,3:]

,Age_08_04,KM,HP,Automatic,CC,Doors,Gears,Weight
0,23,46986,90,0,2000,3,5,1165
1,23,72937,90,0,2000,3,5,1165
2,24,41711,90,0,2000,3,5,1165
3,26,48000,90,0,2000,3,5,1165
4,30,38500,90,0,2000,3,5,1170
...,...,...,...,...,...,...,...,...
1431,69,20544,86,0,1300,3,5,1025
1432,72,19000,86,0,1300,3,5,1015
1433,71,17016,86,0,1300,3,5,1015
1434,70,16916,86,0,1300,3,5,1015


#### Ridge and Lasso with alpha = 10000

In [170]:
# Run ridge regression with penalty equals to 10000
ridge_new = Ridge(alpha=10000)
ridge_model_new = ridge_new.fit(X_train,y_train)

# Print the coefficients
ridge_model.coef_


array([-255.49853171, -162.00766784,   79.02586815,    7.51148239,
         41.06424943,   49.93896511,   11.84737094,  181.0063638 ])

In [172]:

# Run LASSO with penalty = 10000
lasso_new = Lasso(alpha=10000)
lasso_model_new = lasso.fit(X_train,y_train)

# Generate the prediction value from the test data
y_test_pred4 = lasso_model_new.predict(X_test)

# Calculate the MSE
lasso_mse_new = mean_squared_error(y_test, y_test_pred4)
print("Test MSE using lasso with penalty of 10000 = "+str(lasso_mse))

# Print the coefficients
print(lasso_model.coef_)

Test MSE using lasso with penalty of 10000 = 1627430.565671692
[-2323.58993     -721.55495729   434.74989115    85.36114513
    -3.00538181     0.            80.09478236   910.54914831]
